In [1]:
from sklearn.model_selection import StratifiedKFold
from datasets import load_from_disk

import sys
sys.path.append("../code")
from dataloader import PaperDataset
from config import DataArguments, TrainerArguments

In [2]:
data_args = DataArguments
training_args = TrainerArguments

data_args.data_dir = "../data/"
data_args.seed = training_args.seed

In [3]:
dataset = PaperDataset(config=data_args)

In [7]:
type(dataset.dataset)

dict

In [10]:
n_splits = int(1 / data_args.init_pct)
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [11]:
next(kfold.split(dataset.full_dataset["title"], dataset.full_dataset["area"]))

(array([    0,     1,     2, ..., 47247, 47248, 47249]),
 array([   17,    38,    61, ..., 47205, 47218, 47229]))